# extracting data fromthe harvested pages

In [1]:
import pathlib
from lxml import etree
from bs4 import BeautifulSoup
import collections
import csv
import json
import re

In [2]:
sourcefiles = list(pathlib.Path('search pages').glob('*.html'))

In [7]:
ns = {'html': 'http://www.w3.org/1999/xhtml' }
def get_tax_id(node):
    result = node.xpath('./html:a/@href',  namespaces = ns)
#     print(node, result)
    return result[0].split("id=")[-1]

def get_result_name(node):
    result_name = clean_title(node.xpath('./html:a//text()',  namespaces = ns))
    return result_name

def clean_title(titlelist):
    if titlelist:
        return " ".join(" ".join(titlelist).split())
    else:
        return "No result"

def process_record(node, f):
    return [f.name, get_tax_id(node), get_result_name(node)]

def get_extra_tax_ids(resultnode):
    ids = [get_tax_id(r) for r in resultnode]
    return ids

def get_num_results(maxresult, tlen):
    if maxresult:
        mr = int(maxresult[0])
        if mr > int(tlen):
            numresults = str(mr)
        else:
            numresults = tlen
    else:
        numresults = tlen
        
    return numresults

totalresults = re.compile(r'([0-9]+)$')

taxids_to_crawl = []
warn_icon_messages = []

# double checking

zero_results = []
only1 = []
multiple_ok = []
warningitems = []

extra_tax_ids = {'no_warnings': {}, 'major_warnings': []}

with open('datadump.json', 'r') as fin:
    alldata = json.load(fin)
    
for f in sourcefiles[:100]:
    if f.name == '.html':
        continue
    text = f.read_bytes()
    soup = BeautifulSoup(text, 'lxml')
    fixed = soup.prettify()
    root = etree.fromstring(str.encode(fixed)) #wtf this is converting it to bytes?
    rprts = root.xpath('//html:div[@class = "rprt"]', namespaces = ns)
    titles = root.xpath('//html:p[@class = "title"]', namespaces = ns)
    l = len(rprts)
    tlen = len(titles)
    
    item_count_report = root.xpath('//html:h3[@class = "result_count left"]/text()', namespaces = ns)
    maxresult = re.findall(totalresults, " ".join(item_count_report).strip())
    numresults = get_num_results(maxresult, tlen)
    
    warn_icon = root.xpath('//html:li[@class = "warn icon"]/html:span/text()', namespaces = ns)

    alldata[f.name]['num_results'] = numresults
    
    
    if numresults == 0:
        zero_results.append([f,l])
    elif len(warn_icon) == 0:
        if numresults == 1:
            only1.append([f,l])
            result = process_record(titles[0], f)
            taxids_to_crawl.append(result)
            alldata[f.name]['detected_name'] = get_result_name(titles[0])
        else:
            multiple_ok.append([f,l])
    elif "The following term was ignored:" in warn_icon[0]:
        if numresults == 1: 
            only1.append([f,l])
            result = process_record(titles[0], f)
            taxids_to_crawl.append(result)
            alldata[f.name]['detected_name'] = get_result_name(titles[0])
        else:
            multiple_ok.append([f,l])
            alldata[f.name]['detected_name'] = "Multiple Results"
    else:
        warningitems.append([f,l])
        alldata[f.name]['detected_name'] = "Multiple Results"

In [45]:
print(len(zero_results))
print(len(only1))
print(len(multiple_ok))
print(len(warningitems))

498
468
140
469


In [7]:
with open('taxidstocrawl.csv', 'w') as fout:
    csvout = csv.writer(fout)
    csvout.writerow(['taxid', 'name'])
    csvout.writerows(taxids_to_crawl)

In [5]:
alldata

{'ATCC_27511_0000.html': {'query': '(ATCC 27511)',
  'full_path': 'search pages/ATCC_27511_0000.html',
  'sanitisedname': 'ATCC_27511_0000'},
 'BL21_0001.html': {'query': '(BL-21)',
  'full_path': 'search pages/BL21_0001.html',
  'sanitisedname': 'BL21_0001'},
 '2518_0002.html': {'query': '+2518',
  'full_path': 'search pages/2518_0002.html',
  'sanitisedname': '2518_0002'},
 '2711_0003.html': {'query': '+2711',
  'full_path': 'search pages/2711_0003.html',
  'sanitisedname': '2711_0003'},
 '6Fc_IHIL6Fc_and_IL6sIL6R_stimulated_cells_0004.html': {'query': '-6-Fc, I-H-IL-6-Fc and IL-6/sIL-6R stimulated cells',
  'full_path': 'search pages/6Fc_IHIL6Fc_and_IL6sIL6R_stimulated_cells_0004.html',
  'sanitisedname': '6Fc_IHIL6Fc_and_IL6sIL6R_stimulated_cells_0004'},
 'AAD–_cell_0005.html': {'query': '-AAD– cell',
  'full_path': 'search pages/AAD–_cell_0005.html',
  'sanitisedname': 'AAD–_cell_0005'},
 'HAC_0006.html': {'query': '-HAC',
  'full_path': 'search pages/HAC_0006.html',
  'sanitisedn

In [22]:
data

{'ATCC_27511_0000.html': {'query': '(ATCC 27511)',
  'full_path': 'search pages/ATCC_27511_0000.html',
  'sanitisedname': 'ATCC_27511_0000',
  'tax_id': '329',
  'name': 'Ralstonia pickettii'},
 'BL21_0001.html': {'query': '(BL-21)',
  'full_path': 'search pages/BL21_0001.html',
  'sanitisedname': 'BL21_0001',
  'tax_id': '349937',
  'name': 'Clostridium sp. BL-21'},
 '2518_0002.html': {'query': '+2518',
  'full_path': 'search pages/2518_0002.html',
  'sanitisedname': '2518_0002'},
 '2711_0003.html': {'query': '+2711',
  'full_path': 'search pages/2711_0003.html',
  'sanitisedname': '2711_0003'},
 '6Fc_IHIL6Fc_and_IL6sIL6R_stimulated_cells_0004.html': {'query': '-6-Fc, I-H-IL-6-Fc and IL-6/sIL-6R stimulated cells',
  'full_path': 'search pages/6Fc_IHIL6Fc_and_IL6sIL6R_stimulated_cells_0004.html',
  'sanitisedname': '6Fc_IHIL6Fc_and_IL6sIL6R_stimulated_cells_0004'},
 'AAD–_cell_0005.html': {'query': '-AAD– cell',
  'full_path': 'search pages/AAD–_cell_0005.html',
  'sanitisedname': 'AAD

In [26]:
with open('datadump.json', 'r') as fin:
    data = json.load(fin)
    
for fname, taxid, name in taxids_to_crawl:
    data[fname]['tax_id'] = taxid
    data[fname]['name'] = name
def add_error_message(f, message):
    if f.name.startswith('.'):
        print('skipping', f)
    else:
        data[f.name]['group message'] = message

for e, l in empty_v2:
    add_error_message(e, "empty_results")
    data[e.name]['num_results'] = l
    
for e, l in morethan20v2:
#     print(e)
    add_error_message(e, "more_than_20_results")
    if e.name == ".html":
        continue
    data[e.name]['num_results'] = l
    
for e, l in just1accepted:
    add_error_message(e, "only_one_some_errors_accepted")
    data[e.name]['num_results'] = l
    
for e, l in just_1_with_warnings:
    add_error_message(e, "only_one_with_major_warnings")
    data[e.name]['num_results'] = l
    
for e, l in other_multiple_no_warnings:
    add_error_message(e, "multiple_no_warnings")
    data[e.name]['num_results'] = l
    
for e, l in other_muliple_with_warnings:
    add_error_message(e,  "multiple_with_warnings")
    data[e.name]['num_results'] = l

print(len(empty_v2), "totally empty")
print(len(morethan20v2), "have more than 20 resuts")
print(len(just1accepted), "one and only one, with a few warnings accepted")
print(len(just_1_with_warnings), "had one result with major warning")
print(len(other_multiple_no_warnings), "had > 1, <20, no warnings")
print(len(other_muliple_with_warnings), "had > 1, <20 with warnings")
print(len(others))

skipping search pages/.html


KeyError: '.html'

In [15]:
with open('datadump.json', 'w') as fout:
    json.dump(data, fout)


In [10]:
empty_v2

[[PosixPath('search pages/Streptomyces_erythraea_NRRL2338_1098.html'), 0],
 [PosixPath('search pages/SIECΔ_flhDC_0585.html'), 0],
 [PosixPath('search pages/coil_BL21DE3_cells_0718.html'), 0],
 [PosixPath('search pages/EcDL117_0239.html'), 0],
 [PosixPath('search pages/HeLa_0377.html'), 0],
 [PosixPath('search pages/Z_mobilis_ethanologenic_1402.html'), 0],
 [PosixPath('search pages/G108S_0285.html'), 0],
 [PosixPath('search pages/SIECΔp1_0586.html'), 0],
 [PosixPath('search pages/BJ5465_0077.html'), 0],
 [PosixPath('search pages/Luria–Bertani_LB_0468.html'), 0],
 [PosixPath('search pages/Ppu3159_0546.html'), 0],
 [PosixPath('search pages/GFPexpressing_HATresistant_transfectants_0292.html'),
  0],
 [PosixPath('search pages/N_meningitides_0754.html'), 0],
 [PosixPath('search pages/W3110\xa0IJAHFEBCgA_0650.html'), 0],
 [PosixPath('search pages/mutant_SUKA17_transformants_0772.html'), 0],
 [PosixPath('search pages/HDH6D12fruR_0354.html'), 0],
 [PosixPath('search pages/Redαβ_0930.html'), 0],